### Simulation with events

In the [previous](./starting_your_first_simulation.ipynb) notebook we showed you how to start a 3Di simulation. In this notebook
we'll continue with the simulation by adding several events:
- inital waterlevels
- rain
- breaches
- laterals

Let's quickly build a simulation again:

In [ ]:
from datetime import datetime
from getpass import getpass
from openapi_client import ThreedimodelsApi, SimulationsApi
from threedi_api_client.threedi_api_client import ThreediApiClient

In [ ]:
# Provide authentication details
API_HOST = "https://api.3di.live/v3.0"
USERNAME = input("Username")
PASSWORD = getpass("Password")

config = {
    "API_HOST": API_HOST,
    "API_USERNAME": USERNAME,
    "API_PASSWORD": PASSWORD
}

api_client = ThreediApiClient(config=config)

In [ ]:
# Specify the threedi-model and organisation
organisation_uuid = "b08433fa47c1401eb9cbd4156034c679"
models_api = ThreedimodelsApi(api_client)
threedimodel = models_api.threedimodels_list(name__icontains='v2_bergermeer').results[0]

In [ ]:
simulation_api = SimulationsApi(api_client)

simulation = simulation_api.simulations_create(
    data={
        "name": "my first simulation",
        "threedimodel": threedimodel.id,
        "organisation": organisation_uuid,
        "start_datetime": datetime.now(),
        "duration": 3600  # in seconds, so we simulate for 1 hour
    }
)

print(simulation)

Now that we have created a simulation again, we can start adding some interesting events to
the simulation before starting it.

### initial waterlevels

Let's set the waterlevel to 0.5m

In [ ]:
waterlevel = simulation_api.simulations_initial1d_water_level_constant_create(
    simulation_pk=simulation.id, data={"value": 0.5}
)

### Rain

Let's make a time varying rain: every 15 minutes it will decrease in intensity.

In [ ]:
rain = simulation_api.simulations_events_rain_timeseries_create(
    simulation_pk=simulation.id, data={
        'offset':0,
        'values': [[0, 0.005], [900, 0.002], [1800, 0.001], [2700, 0.0005], [3600, 0]],
        'units': 'm/s'}
)

### Breaches

Breaches are dependent on the selected model. So let's first see if the model has
any breaches configured and if so use the first one.

In [ ]:
potential_breaches = models_api.threedimodels_potentialbreaches_list(threedimodel.id)
print(potential_breaches)

# Let's pick the first potential breach
breach = potential_breaches.results[0]

breach_event = simulation_api.simulations_events_breaches_create(
    simulation.id, data={
        "potential_breach": breach.id,
        "duration_till_max_depth": 1800,
        "initial_width": 0.5,
        "offset": 1800
    }
)

### Laterals

We'll add a lateral by specifying a connection_node on which we want the lateral to apply.

In [ ]:
lateral = simulation_api.simulations_events_lateral_constant_create(
    simulation.id, data={
        "offset": 900,
        "connection_node": 4,
        "value": 0.001,
        "duration": 300,
        "units": "m3/s"
    }
)

### Events overview

To get an overview of all our created events on this simulation:

In [ ]:
events = simulation_api.simulations_events(simulation.id)
print(events)

Now let's start the simulation:

In [ ]:
simulation_api.simulations_actions_create(simulation.id, data={"name": "start"})

And eventually the simulation will finish!

In [ ]:
simulation_api.simulations_status_list(simulation.id)